In [2]:
import os
import sys
import pandas as pd
import requests
from lxml import etree
import time

In [3]:
# see
# https://github.com/guigolab/geneid-parameter-files
# for an example on how to set up this script
parameter_files_matrix = "https://raw.githubusercontent.com/guigolab/geneid-parameter-files/main/matrix.tsv"
parameter_files_location = "https://raw.githubusercontent.com/guigolab/geneid-parameter-files/main/parameter_files"

In [4]:
# Define an alternative in case everything fails
selected_param = "Homo_sapiens.9606.param"

data = pd.read_csv(parameter_files_matrix, sep = "\t")

def split_n_convert(x):
    return [int(i) for i in x.replace("'", "").strip("[]").split(", ")]

In [5]:
data.loc[:,"taxidlist"] = data.loc[:,"taxidlist"].apply(split_n_convert)

###
# Separate the lineages into a single taxid per row
###
exploded_df = data.explode("taxidlist")
exploded_df.columns = ["species", "taxid_sp", "parameter_file", "taxid"]
exploded_df.loc[:,"taxid"] = exploded_df.loc[:,"taxid"].astype(int)


NCBI_RESPONSE_MAPPER = {
    'lineage': 'taxidlist',
    'tax_id':'taxid',
    'organism_name':'species'
}

In [6]:
exploded_df

,species,taxid_sp,parameter_file,taxid
0,Plasmodium vivax,5855,Plasmodium_vivax.5855.param,5855
0,Plasmodium vivax,5855,Plasmodium_vivax.5855.param,418103
0,Plasmodium vivax,5855,Plasmodium_vivax.5855.param,5820
0,Plasmodium vivax,5855,Plasmodium_vivax.5855.param,1639119
0,Plasmodium vivax,5855,Plasmodium_vivax.5855.param,5819
...,...,...,...,...
63,Parastagonospora nodorum,13684,Stagonospora_nodorum.13684.param,4751
63,Parastagonospora nodorum,13684,Stagonospora_nodorum.13684.param,33154
63,Parastagonospora nodorum,13684,Stagonospora_nodorum.13684.param,2759
63,Parastagonospora nodorum,13684,Stagonospora_nodorum.13684.param,131567


In [7]:
def get_taxon_from_ENA(taxid):
    taxon = dict()
    try:
        response = requests.get(f"https://www.ebi.ac.uk/ena/browser/api/xml/{taxid}")
        xml = response.content
        if not xml or len(xml) == 0:
            return taxon
        root = etree.fromstring(xml)
        for child in root:
            child_taxid = int(child.attrib["taxId"])
            if(child_taxid == taxid):
                lineage = []
                for taxon_node in child:
                    if taxon_node.tag == 'lineage':
                        lineage.append(int(child.attrib["taxId"]))
                        lineage.extend([int(node.attrib['taxId']) for node in taxon_node])
                        taxon['lineage'] = lineage
                        taxon['taxid'] = child_taxid
                        taxon['species'] = child.attrib['scientificName']
                        break
        return taxon
    except:
        return taxon

In [8]:
    
def get_taxon_from_NCBI(taxid):
    taxon = dict()
    try:
        response = requests.get(f"https://api.ncbi.nlm.nih.gov/datasets/v1/taxonomy/taxon/{taxid}")
        if not response.json() or not 'taxonomy_nodes' in response.json():
            return taxon
        taxon_to_insert = response.json()['taxonomy_nodes'][0]['taxonomy']
        for k in NCBI_RESPONSE_MAPPER.keys():
            value = NCBI_RESPONSE_MAPPER[k]
            taxon[value] = taxon_to_insert[k]
        return taxon
    except:
        return taxon

In [9]:
def get_lineage_from_taxid(new_taxid):
    counter = 0
    if counter > 2:
        time.sleep(1)
    taxon = get_taxon_from_NCBI(new_taxid)
    if not taxon:
        taxon = get_taxon_from_ENA(new_taxid)
        if not taxon:
            counter += 1
            #skip taxon if does not exists in ncbi
            return None
    if taxon['taxidlist'][0] == 1:
        taxon['taxidlist'].reverse()
    return taxon['taxidlist']

In [12]:
Trifolium_dubium = (97021, 'Trifolium_dubium')
Homo_sapiens = (9606, 'Homo_sapiens')
Drosophila_melanogaster = (7227,'Drosophila_melanogaster')
Helleia_helle = (2795559,'Helleia_helle')
Phakellia_ventilabrum = (942649,'Phakellia_ventilabrum')
Pocillopora_meandrina = (46732 , 'Pocillopora_meandrina')

species_list = [Trifolium_dubium,
                Homo_sapiens,
                Drosophila_melanogaster, 
                Helleia_helle, 
                Phakellia_ventilabrum,
                Pocillopora_meandrina]

In [16]:
for s in species_list:
    
    target_taxid = s[0]
    species_name = s[1]
    
    # taxid of interest
    target_lineage = get_lineage_from_taxid(target_taxid)

    ###
    # Get the species of interest lineage
    ###

    query = pd.DataFrame(target_lineage)
    query.columns = ["taxid"]
    query.loc[:,"taxid"] = query.loc[:,"taxid"].astype(int)

    ###
    # Intersect the species lineage with the dataframe of taxids for parameters
    ###
    intersected_params = query.merge(exploded_df, on = "taxid")

    ###
    # If there is an intersection, select the parameter whose taxid appears
    #   less times, less frequency implies more closeness
    ###
    if intersected_params.shape[0] > 0:

        taxid_closest_param = intersected_params.loc[:,"taxid"].value_counts().sort_values().index[0]

        selected_param = intersected_params[intersected_params["taxid"] == taxid_closest_param].loc[:,"parameter_file"].iloc[0]

    # print(target_taxid)
    print(f"{species_name} = {parameter_files_location}", selected_param, sep = "/", end = "\n")
#     print(f"{species_name} = {parameter_files_location}")

Trifolium_dubium = https://raw.githubusercontent.com/guigolab/geneid-parameter-files/main/parameter_files/Phaseolus_vulgaris.3885.param
Homo_sapiens = https://raw.githubusercontent.com/guigolab/geneid-parameter-files/main/parameter_files/Homo_sapiens.9606.param
Drosophila_melanogaster = https://raw.githubusercontent.com/guigolab/geneid-parameter-files/main/parameter_files/Drosophila_melanogaster.7227.param
Helleia_helle = https://raw.githubusercontent.com/guigolab/geneid-parameter-files/main/parameter_files/Dinoponera_longipes.609292.param
Phakellia_ventilabrum = https://raw.githubusercontent.com/guigolab/geneid-parameter-files/main/parameter_files/Dinoponera_longipes.609292.param
Pocillopora_meandrina = https://raw.githubusercontent.com/guigolab/geneid-parameter-files/main/parameter_files/Dinoponera_longipes.609292.param
